In [1]:
import os
cachedir = '/rscratch/tpang/kinshuk/cache'
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
os.environ["TRANSFORMERS_CACHE"] = cachedir
os.environ["HF_DATASETS_CACHE"]= cachedir
from model import get_model
from loader.layers import param_count
from loader.data_module import make_data_module
import json
import torch
import random
import logging
import argparse
import numpy as np
import transformers
from pathlib import Path
import torch.backends.mps
import torch.backends.cudnn
from torch.cuda import (
    max_memory_allocated,
    reset_peak_memory_stats,
    reset_max_memory_allocated,
    memory_allocated,
)
from loader.logger import get_logger
from transformers import set_seed
# from accelerate import Accelerator
from torch.utils.data import DataLoader
from os.path import exists, join, isdir
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence

from transformers.utils.logging import (
    set_verbosity_error as transformers_vb_err,
)
from datasets.utils.logging import (
    set_verbosity_error as datasets_vb_err,
)
from transformers import Seq2SeqTrainer
from traineval.eval import eval_func
logger = logging.getLogger(__name__)
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
from llamaft import ModelArguments, DataArguments, TrainingArguments, GenerationArguments

/rscratch/tpang/kinshuk/anaconda3/envs/kinbert/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-07 04:05:48.441963: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 04:05:48.442005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 04:05:48.443216: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-07 04:05:48.450737: I tensorflow/core/platform/cpu_feature_guard.cc:182] Thi

In [2]:
# Setting up the arguments

model_args = ModelArguments(
    model_name_or_path="meta-llama/Llama-2-7b-hf"
)

data_args = DataArguments(
    eval_dataset_size=100,
    max_eval_samples=1000,
    source_max_len = 1024,
    target_max_len = 256,
    dataset="alpaca", # DATASET [alpaca|chip2|self-instruct|hh-rlhf|oasst1|longform]
)

training_args = TrainingArguments(
    output_dir="./output",
    data_seed=7,
    save_strategy="steps",
    evaluation_strategy="steps",
    logging_strategy="steps",
    do_eval=True,
    eval_steps=187,
    adam_beta2=0.999,

    learning_rate=5e-6,     # LEARNING RATE
    
    max_steps=300,         # NUMBER OF STEPS

    sortby="alpha",         # CAN DO "alpha" or "lora" or "dora"

    num_layers=20,           # NUMBER OF LAYERS FOR FULL FINE-TUNING

    per_device_train_batch_size = 2, # BATCH-SIZE
    memlog=False,
)

generation_args = GenerationArguments(
    max_new_tokens=256 # default is 256
)

# If you need to use GenerationConfig or similar for generation_args
training_args.generation_config = transformers.GenerationConfig(
    **vars(generation_args)
)

# Combine arguments into a single Namespace object (if needed)
args = argparse.Namespace(
    **vars(model_args), **vars(data_args), **vars(training_args),
)

# Control randomness
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
set_seed(args.seed)  # transformers seed

In [3]:
if 'lora' in args.sortby:
    args.num_layers = 0
logger = logging.getLogger(__name__)
gpus = torch.cuda.device_count()
mempath = (
    f"/rscratch/tpang/kinshuk/RpMKin/llama_ft/{args.dataset}/"
    + f"{args.sortby}"
)
start_memory = [0] * gpus
end_memory = [0] * gpus
peek_memory = 0

if args.verbose:
        task_info = (
            f"\nSeed: {args.seed}\n"
            + f"Dataset: {args.dataset}\n"
            + f"Sort by: {args.sortby}\n"
            + f"Layers to train: {args.num_layers}\n"
        )
        print(task_info)
else:
    datasets_vb_err()
    transformers_vb_err()
    global _tqdm_active
    _tqdm_active = False


Seed: 7
Dataset: alpaca
Sort by: alpha
Layers to train: 20



In [4]:
def memall(gpus=gpus):
    for i in range(gpus):
        start_memory[i] = torch.cuda.memory_allocated(i)
    return sum(start_memory)

model, tokenizer = get_model(args)

for device in range(gpus):
    reset_peak_memory_stats(device=device)
    reset_max_memory_allocated(device=device)

weight_memory = memall()

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.69s/it]


Adding special tokens.
Sorted by  alpha
Training layers: ['model.layers.21.self_attn.v_proj', 'model.layers.22.self_attn.v_proj', 'model.layers.16.self_attn.v_proj', 'model.layers.19.self_attn.v_proj', 'model.layers.18.self_attn.v_proj', 'model.layers.30.self_attn.v_proj', 'model.layers.28.self_attn.v_proj', 'model.layers.17.self_attn.v_proj', 'model.layers.27.self_attn.v_proj', 'model.layers.11.self_attn.v_proj', 'model.layers.23.self_attn.o_proj', 'model.layers.3.mlp.up_proj', 'model.layers.21.self_attn.o_proj', 'model.layers.25.mlp.up_proj', 'model.layers.24.self_attn.o_proj', 'model.layers.20.self_attn.v_proj', 'model.layers.26.mlp.up_proj', 'model.layers.27.self_attn.o_proj', 'model.layers.25.self_attn.o_proj', 'model.layers.22.self_attn.o_proj']
Enabling model.layers.3.mlp.up_proj.weight parameter
Enabling model.layers.11.self_attn.v_proj.weight parameter
Enabling model.layers.16.self_attn.v_proj.weight parameter
Enabling model.layers.17.self_attn.v_proj.weight parameter
Enabling

In [5]:
data_module = make_data_module(tokenizer=tokenizer, args=args) # type: ignore
dataset = {k:v for k,v in data_module.items()}
train_dataloader = DataLoader(
    dataset['train_dataset'], # type: ignore
    batch_size=args.per_device_train_batch_size,
    collate_fn=dataset['data_collator'],
    shuffle=True,
)
train_dataloader.__len__()

Splitting train dataset in train and validation according to `eval_dataset_size`


In [8]:
def loss_fn(x, y):
    "A Flat CrossEntropy" 
    return torch.nn.functional.cross_entropy(x.view(-1, x.shape[-1]), y.view(-1))

from tqdm.auto import tqdm
train_losses = []
val_losses = []
val_accs = []

model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
optimizer.zero_grad()
optimizer_memory = 0

for epoch in range(1):
    train_loss = 0
    tr_steps = 0
    for step, batch in enumerate((train_dataloader)):

        optimizer.zero_grad()
        curr = memall()
        batch = {k: v.to(model.device) for k, v in batch.items()}
        input_memory = memall() - curr
        
        curr = memall()
        output = model(**batch)
        activation_memory = memall() - curr
        
        curr = memall()
        # loss = loss_fn(out.logits, batch["labels"]) / args.gradient_accumulation_steps
        loss = output.loss
        loss.backward()
        gradient_memory = memall() - input_memory - weight_memory - optimizer_memory

        curr = memall()
        optimizer.step()
        if step == 0:
             optimizer_memory = memall() - curr
    
        loss = loss.cpu()
        train_loss += loss.item()
        tr_steps += 1
        train_losses.append(train_loss/tr_steps)
        if step % 1 == 0:
            print(f'Step: {step}, Train Loss: {train_loss/tr_steps}')
        torch.cuda.empty_cache()
        if step == args.max_steps:
            model.eval()
            break

total_memory = memall()

Step: 0, Train Loss: 1.6104059219360352
Step: 1, Train Loss: 2.7800209522247314
Step: 2, Train Loss: 2.3085394303003945
Step: 3, Train Loss: 2.010852873325348
Step: 4, Train Loss: 1.902347159385681
Step: 5, Train Loss: 1.7808004021644592
Step: 6, Train Loss: 1.71690377167293
Step: 7, Train Loss: 1.630031555891037
Step: 8, Train Loss: 1.6000707546869914
Step: 9, Train Loss: 1.6106316924095154
Step: 10, Train Loss: 1.6683146845210681
Step: 11, Train Loss: 1.625487208366394
Step: 12, Train Loss: 1.6004160826022809
Step: 13, Train Loss: 1.5283176260335105
Step: 14, Train Loss: 1.5391154170036316
Step: 15, Train Loss: 1.5425435416400433
Step: 16, Train Loss: 1.5110683686593
Step: 17, Train Loss: 1.5000833769639332
Step: 18, Train Loss: 1.492152656379499
Step: 19, Train Loss: 1.4621799647808076
Step: 20, Train Loss: 1.4478481270018078
Step: 21, Train Loss: 1.4306446476416155
Step: 22, Train Loss: 1.4187893452851668
Step: 23, Train Loss: 1.4300648073355358
Step: 24, Train Loss: 1.422753047943

In [9]:
optimizer.zero_grad()
model.eval()
trainer=Seq2SeqTrainer(
                model=model,
                tokenizer=tokenizer,
                args=training_args,
                **{k:v for k,v in data_module.items() if k != 'predict_dataset'},
            )
all_metrics = {"run_name": args.run_name}
if args.do_eval:
    all_metrics = eval_func(args, logger, trainer, all_metrics)
total_memory = memall()
peek_memory = sum([max_memory_allocated(i) for i in range(gpus)])


INFO:__main__:*** Evaluate ***


***** eval metrics *****
  eval_loss               =     1.0292
  eval_runtime            = 0:00:05.94
  eval_samples_per_second =      16.83
  eval_steps_per_second   =      2.188


In [10]:
all_metrics

{'run_name': './output',
 'eval_loss': 1.0291874408721924,
 'eval_runtime': 5.9418,
 'eval_samples_per_second': 16.83,
 'eval_steps_per_second': 2.188}

In [11]:
memory_string = (
        f"{param_count(model)}\n"
        f"Method           : {args.sortby}\n"
        f"Layers           : {args.num_layers}\n"
        f"Learning Rate    : {args.learning_rate}\n"
        f"Batch size       : {args.per_device_train_batch_size}\n"
        f"Weight memory    : {weight_memory / 1e6} MB\n"
        f"Optimizer memory : {optimizer_memory / 1e6} MB\n"
        f"Activation memory: {activation_memory / 1e6} MB\n"
        f"Gradient memory  : {gradient_memory / 1e6} MB\n"
        f"Input memory     : {input_memory / 1e6} MB\n"
        f"Total memory     : {total_memory / 1e6} MB\n"
        f"Peak memory      : {peek_memory / 1e6} MB\n"
    )
print(memory_string)

Total params: 6738.42M, Trainable: 551.56M
(6738.423808, 551.555072)
Method           : alpha
Layers           : 20
Learning Rate    : 5e-06
Batch size       : 2
Weight memory    : 27087.929344 MB
Optimizer memory : 4412.440576 MB
Activation memory: 1312.103424 MB
Gradient memory  : 2245.021184 MB
Input memory     : 0.003584 MB
Total memory     : 31539.1744 MB
Peak memory      : 38131.032064 MB



In [12]:
# SAVE TRAINING HISTORY
base = {"train_loss": train_loss,}
savepath = f"./output/{args.dataset}/lr_{args.learning_rate}/batch_{args.per_device_train_batch_size}/{args.sortby}/layers_{args.num_layers}"
if args.memlog:
    Path(savepath).mkdir(parents=True, exist_ok=True)
    np.save(os.path.join(savepath, "finetune.npy"), base) # type: ignore
    with open(os.path.join(savepath, "metrics.json"), "w") as fout:
        fout.write(json.dumps(all_metrics))
    log_info = (
        f"\n\n{args.dataset} "
        + f"Batch Size {args.per_device_train_batch_size} "
        + f"{args.sortby} fine-tuning "
        + f"{args.num_layers} Layers"
    )
    logger = get_logger(savepath, "memlog.log")
    logger.info(log_info)
    logger.info(f"\n{memory_string}\n")
    if (args.do_train or args.do_eval or args.do_predict):
        with open(os.path.join(savepath, "metrics.json"), "w") as fout:
            fout.write(json.dumps(all_metrics))